In [ ]:
!pip install praw
!pip install SQLAlchemy
!pip install openai


combine the title and text of each submission id for a dictionary and also initialize the overall setup

In [12]:
import datetime as dt
from sqlalchemy import create_engine, Column, String, Integer, Text
from sqlalchemy.orm import declarative_base, sessionmaker
import os
from openai import AzureOpenAI

# SQLAlchemy setup
DATABASE_URL = "sqlite:////content/sites_crawled(reddit)9.db3"
engine = create_engine(DATABASE_URL)
Base = declarative_base()

class Submission(Base):
    __tablename__ = 'submissions'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    text = Column(Text)
    author = Column(String)
    url = Column(String)
    posted_at = Column(String)
    edited_at = Column(String, nullable=True)
    votes = Column(Integer)

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Retrieve data from the database
submissions = session.query(Submission).all()

# Create a dictionary where each key is the submission ID and the value is the combined title and text, both converted to lowercase
submission_data = {}
for submission in submissions:
    combined_text = submission.title.lower() + ' ' + submission.text.lower()
    submission_data[submission.id] = combined_text

# Close the session
session.close()

# Now submission_data is a dictionary where each key is the submission ID and the value is the combined title and text of the submission, both converted to lowercase


In [13]:
print(submission_data)

{1: 'roofer recommendations in the stony brook area. alright redditors, looking for a roofer recommendation in the stony brook area.  thanks!', 2: "news 12 coverage  of sbu protests has been hilarious... just to preface this isn't a comment on what people are protesting for or against, or if they are impacting anything or not, it's a comment on news 12.\n\ngearing up for a trip back to the island, what better way to get in the head space than watch some news 12? i've had it on all day working, from morning to 6pm at least they have been going to a reporter at stony brook university really hamming it up on campus talking about a group of students protesting. all day it's been like 14 students, and you can tell by the reporter's comments they want this to be a version of columbia's protests. this reporter has been frothing at the mouth with excitement talking in front of 14 people all day who are just hanging out basically like it's some normal day.\n\nedit: there are more people joining

using openai is each id to text above related to healthcare


In [ ]:
import os
from openai import AzureOpenAI, BadRequestError

# Initialize the AzureOpenAI client with the provided Azure OpenAI endpoint, API key, and API version
client = AzureOpenAI(
    azure_endpoint="https://azopenai-bmires.openai.azure.com/",
    api_key="78c4753585944336b17a0d9a642a4f1e",
    api_version="2024-02-15-preview"
)

# Initialize an empty list to store submission IDs where the completion response indicates "yes"
yes_submission_ids = []

for submission_id, combined_text in submission_data.items():
    try:
        message_text = [{"role": "system", "content": "Can you tell me if this content is related to healthcare(Only Humans)? Output yes or no: " + combined_text}]
        completion = client.chat.completions.create(
            model="BMI_EHRnotes",
            messages=message_text,
            temperature=0.7,  # Controls the randomness of the output
            max_tokens=800,  # Maximum number of tokens (words) in the completion
            top_p=0.95,  # Controls the nucleus sampling top probability
            frequency_penalty=0,  # Adjusts for word frequency in the output
            presence_penalty=0,  # Adjusts for repetition in the output
            stop=None  # Token at which to stop generation, or None for no stop token
        )
        completion_text = completion.choices[0].message.content.lower()
        print(f"Submission ID: {submission_id}, Completion: {completion_text}")
        if "yes" in completion_text:
            yes_submission_ids.append(submission_id)
    except BadRequestError as e:
        print(f"Skipping submission ID {submission_id} due to content filtering policies.")
        continue  # Skip to the next submission in case of content filtering error

# Print all the IDs where the completion response indicates "yes"
print("Submission IDs where the completion response indicates 'yes':", yes_submission_ids)

# Save the IDs to a text file
with open("yes_submission_ids.txt", "w") as file:
    for submission_id in yes_submission_ids:
        file.write(str(submission_id) + "\n")


In [ ]:
arr = [5, 6, 7, 14, 15, 16, 17, 18, 27, 34, 41, 43, 45, 48, 49, 52, 55, 58, 59, 60, 61, 62, 64, 69, 72, 75, 84, 87, 89, 95, 99, 100, 101, 103, 106, 120, 136, 141, 152, 159, 160, 167, 181, 184, 188, 189, 190, 197, 198, 199]

Finding if the text is related to sbu or northwell health

In [19]:
import os
from openai import AzureOpenAI, BadRequestError

# Initialize the AzureOpenAI client with the provided Azure OpenAI endpoint, API key, and API version
client = AzureOpenAI(
    azure_endpoint="https://azopenai-bmires.openai.azure.com/",
    api_key="78c4753585944336b17a0d9a642a4f1e",
    api_version="2024-02-15-preview"
)

# Initialize an empty list to store submission IDs where the completion response indicates "yes"
yes_submission_ids = []

# Array of specific numbers to run the prompt for
arr = [4, 5, 6, 7, 14, 15, 16, 17, 18, 27, 34, 41, 43, 45, 48, 49, 52, 55, 58, 59, 60, 61, 62, 64, 69, 72, 75, 84, 87, 89, 95, 98, 99, 100, 101, 103, 106, 120, 136, 141, 152, 159, 160, 167, 181, 184, 188, 189, 190, 196, 197, 198, 199]

for submission_id, combined_text in submission_data.items():
    if submission_id in arr:
        try:
            message_text = [{"role": "system", "content": "Can you tell me if this content is related to Northwell health or Stony Brook Univ or neither, if Northwell return NH, if sbu return SBU if both return both else return Neither " + combined_text}]
            completion = client.chat.completions.create(
                model="BMI_EHRnotes",  # Specify the model (deployment_name)
                messages=message_text,
                temperature=0.7,  # Controls the randomness of the output
                max_tokens=800,  # Maximum number of tokens (words) in the completion
                top_p=0.95,  # Controls the nucleus sampling top probability
                frequency_penalty=0,  # Adjusts for word frequency in the output
                presence_penalty=0,  # Adjusts for repetition in the output
                stop=None  # Token at which to stop generation, or None for no stop token
            )
            completion_text = completion.choices[0].message.content.lower()
            print(f"Submission ID: {submission_id}, Completion: {completion_text}")
            yes_submission_ids.append(submission_id)
        except BadRequestError as e:
            print(f"Skipping submission ID {submission_id} due to content filtering policies.")
            continue  # Skip to the next submission in case of content filtering error




Skipping submission ID 4 due to content filtering policies.
Submission ID: 5, Completion: both
Submission ID: 6, Completion: neither
Submission ID: 7, Completion: sbu
Submission ID: 14, Completion: sbu
Submission ID: 15, Completion: sbu
Submission ID: 16, Completion: sbu
Submission ID: 17, Completion: sbu
Submission ID: 18, Completion: sbu
Submission ID: 27, Completion: sbu
Submission ID: 34, Completion: both
Submission ID: 41, Completion: sbu
Submission ID: 43, Completion: both
Submission ID: 45, Completion: sbu
Submission ID: 48, Completion: sbu
Submission ID: 49, Completion: sbu
Submission ID: 52, Completion: sbu
Submission ID: 55, Completion: neither
Submission ID: 58, Completion: neither
Submission ID: 59, Completion: neither
Submission ID: 60, Completion: neither
Submission ID: 61, Completion: neither
Submission ID: 62, Completion: neither
Submission ID: 64, Completion: neither
Submission ID: 69, Completion: neither
Submission ID: 72, Completion: neither
Submission ID: 75, Comple